In [4]:
import os
import shutil

# 元のフォルダパスと新しいフォルダのパスを指定
original_folder = '/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/split_files/group_3'
new_folder = '/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/split_files/group_3_1'

# 新しいフォルダを作成
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

# 元のフォルダ内のファイルのリストを取得
file_list = os.listdir(original_folder)
file_count = len(file_list)

# ファイルの数を5つのグループに均等に分割
groups = [file_list[i::3] for i in range(3)]  # 5つのグループに分割

# 各グループのフォルダを作成し、ファイルをコピー
for idx, group in enumerate(groups, start=1):
    group_folder = os.path.join(new_folder, f'group_{idx}')  # 新しいグループのフォルダを作成
    os.makedirs(group_folder)

    # ファイルを新しいフォルダにコピー
    for file in group:
        src = os.path.join(original_folder, file)
        dst = os.path.join(group_folder, file)
        shutil.copy(src, dst)


In [3]:
# 何回目すか,,マジで
import pandas as pd 
import os
import datetime
import numpy as np
from datetime import timedelta 
import csv
import time
import math
from scipy.stats import norm
from scipy.optimize import minimize
import random


In [4]:
# inputも時間間隔20secで実行
df_quater = pd.read_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_0845_20sec.csv')
search_folder = "/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/split_files/group_3_1/group_2"
file_list = list(os.listdir(search_folder))

# df_linkも作り直し
df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_link_post.csv')
L = len(df_link)

# df_nodeは作り直し
df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_node.csv')

df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/ble_nw.csv')

In [17]:

###### 各linkのoとdの座標を入れておく配列 ###### # 線分とビーコンの距離に関しての情報は向きがあってもなくても変わらない
oddslink_loc_array = [] 
for i in range(len(df_link)):
    linkid = df_link.loc[i, 'linkid']
    O = df_link.loc[i, 'o']
    D = df_link.loc[i, 'd']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['floor'].iloc[0] 
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['floor'].iloc[0] # これでseriesではなく値として抽出できるはず
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)

    oddslink_loc_array.append(loc_tuple) # oddslink_loc_array[linkid//2][0]がlinkidのonode, [1]がdnode，[2]が中点，，，，

###### 各ビーコンの座標を入れておく配列 ###### 
beacon_loc_array = []
for j in range(len(df_ble)):
    beacon_id = df_ble.loc[j, 'ID']
    x_j = df_ble[df_ble['ID'] == beacon_id]['x'].iloc[0]
    y_j = df_ble[df_ble['ID'] == beacon_id]['y'].iloc[0]
    z_j = df_ble[df_ble['ID'] == beacon_id]['floor'].iloc[0]
    beacon_loc = np.array([x_j, y_j, z_j])

    beacon_loc_array.append(beacon_loc) # beacon_loc_array[i]がビーコンi+1の位置座標

###### ビーコンとリンク線分の中点間距離を返す関数 ######
def shortest_distance_to_segment(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)
    
    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    mid_point = (p1 + p2) / 2

    # 垂直ベクトルを計算
    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
    distance = np.linalg.norm(x - mid_point)
    
    return distance

###### d_arrayの用意 ###### # 各リンク(の中点）とbleビーコンとの距離配列
d_array = np.zeros((len(df_link), len(df_ble)))

for i in range(len(df_link)):
    p_o = oddslink_loc_array[i][0] # o座標 # i=1の時
    p_d = oddslink_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'floor']
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = shortest_distance_to_segment(p_o, p_d, p_ap) # ijはlink i+1, ビーコンj+1の距離

###### 混雑度dens，maxRSSI，meanRSSI，検出回数を各ビーコン各timestepごとに入れた配列（ビーコン数23*タイムステップ数） ######
dens_jt = np.zeros((23, 45), dtype = int)
max_jt = np.zeros((23, 45), dtype = int)
mean_jt = np.zeros((23, 45), dtype = int)
#count_jt = np.zeros((23, 45), dtype = int)

grouped = df_quater.groupby(['ID', 'timestep']) # listにして渡す必要がある
df_list = [group.reset_index(drop=True) for name, group in grouped]

for group in df_list:
    beacon_id = int(group.loc[0, 'ID'])
    timestep = int(group.loc[0, 'timestep'])

    # ユニークMACアドレス数，最大&平均RSSI
    unique_mac_count = group['MAC'].nunique()
    max_rssi = group['RSSI'].max()
    mean_rssi = group['RSSI'].mean()
    # countはそのタイムステップでの全検出数つまりgroupの長さそのもの
    #count = len(group)

    # numpy配列の要素はシーケンスダ
    dens_jt[beacon_id-1, timestep-1] = unique_mac_count
    max_jt[beacon_id-1, timestep-1] = max_rssi
    mean_jt[beacon_id-1, timestep-1] = mean_rssi
    #count_jt[beacon_id-1, timestep-1] = count

####### リンク接続行列（無向） ####### 
def I_nod(link_data): 
    n = len(link_data)
    I = np.eye(n) # 滞在リンク（吸収リンクにもつけて良い）
    for i in range(n):
        O = link_data.loc[i, 'o'] # 当該linkのOnode
        D = link_data.loc[i, 'd'] # 当該linkのDnode
        for j in range(n): # リンクiのOorDをOないしDとしているリンクjとは接続
            if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                I[i, j] = 1
    return(I)

I = I_nod(df_link)

###### 階段リンクについて別途処理 ###### # i = 26~44
for i in range(26, 45, 2):
    # 階段には滞在リンク入れない．向きは考えないので面倒なことは考えない
    I[i-1, i-1] = 0
    I[i, i] = 0

###### 吸収リンクからは吸収リンクにしか接続しない
for i in range(46, 49):
    I[i-1, :] = 0 # 他には繋がらない
    I[i-1, i-1] = 1 # 自分自身にはつながる # 時間構造化するなら自分自身への滞在は可能にする!!!!!とりあえず構造化でできるかチャレンジする

###### 改札の吸収リンク ######
ddata = [46, 47, 48]
D = len(ddata)

####### パラメタ初期設定 #######
xdim = 2
x = np.zeros(xdim)
x[0] = -1
x[-1] = 0.8
sigma = 5 #### 想像できなすぎる．理想は，リンクごとに設定して推定

###### タイムステップ設定 ######
T = 11 # 10stepで最終リンクには辿り着く人たち，吸収に行くのに+1timestep要すると考えるとT=11が妥当か．

####### 観測モデル ######
####### 観測方程式 ######
def qq(dist):
    #return 
    return (dist/((sigma)**2) + np.exp(-((dist)**2)/2/((sigma)**2)))

# 重み付き最小二乗法を最小化する関数
def weighted_least_squares(coordinates, observations):
    # 各観測点の座標と重みを取り出す
    observed_coordinates = observations[:, :-1]  # 最後の列（重み）を除いた座標
    weights = observations[:, -1]  # 重み
    
    # 予測座標と観測座標の誤差の計算
    errors = coordinates - observed_coordinates
    
    # 重みを考慮した重み付き誤差の二乗和を返す
    weighted_errors = weights * np.sum(errors ** 2, axis=1)
    
    return np.sum(weighted_errors)

def measuring_model(x):
    print(f'パラメタ{x}で観測モデル回す')
    measured_loc = np.zeros(6) # 'userid', 'timestep', 'x', 'y', 'z' 
    for file_name in file_list:
        file_path = os.path.join(search_folder, file_name)
        dfi = pd.read_csv(file_path)
        
        base_name = file_name.split('.')[0]   
        userid = int(base_name)

        nts = dfi['timestep'].nunique()                
        grouped = dfi.groupby('timestep')
        dfi_list = [group.reset_index(drop=True) for name, group in grouped]  

        i_res = np.zeros(6)
            
        for t in range(nts): # tが最後の時は次の最大ビーコンの座標を参照できない
            dfit = dfi_list[t]
            ts = int(dfit.loc[0, 'timestep'])

            result_beacon = dfit.groupby('ID')
            result_beacon_list = [group.reset_index(drop=True) for name, group in result_beacon]

            tdict = {}

            for j in range(len(result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                dfitj = result_beacon_list[j]
                jscore = 0 # 初期値は0にしておく
                jid = dfitj.loc[0, 'ID'] # このビーコンのid
                max_rssi = max_jt[jid-1, ts-1] # tsでのbeaconのmaxのrssi

                for k in range(len(dfitj)): 
                    rssi = dfitj.loc[k, 'RSSI']                        
                    jscore += max_rssi / rssi 
                    tdict[jid] = jscore  
            
            # dictにビーコンのidとRSSI実績値による重みが入った．最大ビーコンのidをメモ
            jmax = max(tdict, key = tdict.get)

            if t < nts-1:
                if len(tdict) > 1: # 2点以上で観測されてたら，最小二乗法で一点の推定位置を出すことが可能  
                    observation_array = np.zeros(4)
                    for b in range(len(tdict)):
                        beacon_i = list(tdict.keys())[b] # beacon_iはビーコンのidなので-1してindexにする
                        beacon_i = int(beacon_i)
                        loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
                        beacon_w = list(tdict.values())[b]

                        result_array = np.insert(loc_i, len(loc_i), beacon_w)
                        observation_array = np.vstack((observation_array, result_array))
                    
                    # 先頭の0の行を削除
                    observation_array = np.delete(observation_array, 0, axis = 0)
                    # n点の位置が取得されてobservation_arrayに入った．weighted_least_squares(coordinates, observations)を使って位置を推定

                    # 初期座標の設定（仮の値を使用，ある程度それっぽい値を入れないと変な値を返してくるので注意）
                    initial_coordinates = np.array([-11920, -37880, 20])

                    # 重み付き最小二乗法を最小化して推定座標を求める
                    result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

                    # 推定された座標を取得
                    estimated_coordinates = result.x
                    estimated_coordinates_with_j = np.insert(estimated_coordinates, 0, jmax)
                    estimated_coordinates_with_ts = np.insert(estimated_coordinates_with_j, 0, ts) # tsを列の先頭に追加

                elif len(tdict) == 1: # その時点での観測が1ビーコンcだったら→次の観測を参照
                    observation_array = np.zeros(4)

                    beacon_i = list(tdict.keys())[0] # beacon_iはビーコンのidなので-1してindexにする
                    beacon_i = int(beacon_i)
                    loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
                    beacon_w = list(tdict.values())[0]

                    result_array = np.append(loc_i, float(beacon_w))
                    observation_array = np.vstack((observation_array, result_array))

                    # 次のmaxbeaconを参照
                    dfitt = dfi_list[t+1]
                
                    next_result_beacon = dfitt.groupby('ID')
                    next_result_beacon_list = [group.reset_index(drop=True) for name, group in next_result_beacon]

                    # 時刻t+1の各ビーコンのscoreを記録するdictの準備
                    ttdict = {}
                    for j in range(len(next_result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                        dfittj = next_result_beacon_list[j]
                        next_jscore = 0 # 初期値は0にしておく
                        next_jid = dfittj.loc[0, 'ID'] # このビーコンのid
                        next_max_rssi = max_jt[jid-1, (ts+1)-1] # tsでのbeaconのmaxのrssi

                        for k in range(len(dfittj)): # freq = count / count_tot
                            next_rssi = dfittj.loc[k, 'RSSI']                        
                            next_jscore += next_max_rssi / next_rssi # beacon_eval(rssi, max_rssi) # , freq)
                            ttdict[jid] = next_jscore

                    # 次の最大beaconのkeyとscoreを取得して
                    next_max_key = max(ttdict, key = ttdict.get)
                    next_max_score = ttdict[next_max_key]

                    next_max_beacon_loc = beacon_loc_array[next_max_key-1] # 次の最大ビーコンの座標

                    next_result_array = np.insert(next_max_beacon_loc, len(next_max_beacon_loc), next_max_score)
                    observation_array = np.vstack((observation_array, next_result_array))

                    # 先頭の0の行を削除
                    observation_array = np.delete(observation_array, 0, axis = 0)

                    # 初期座標の設定
                    initial_coordinates = np.array([-11920, -37880, 20])
                    result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

                    # 推定された座標を取得
                    estimated_coordinates = result.x
                    estimated_coordinates_with_j = np.insert(estimated_coordinates, 0, jmax)
                    estimated_coordinates_with_ts = np.insert(estimated_coordinates_with_j, 0, ts) # tsを列の先頭に追加(ts, x, y, z)
            
            elif t == nts-1:
                last_beacon = max(tdict, key = tdict.get)
                last_beacon = int(last_beacon)
                estimated_coordinates = beacon_loc_array[last_beacon-1] # 最後は一番検出が顕著だったビーコンの座標で
                
                estimated_coordinates_with_j = np.insert(estimated_coordinates, 0, jmax)
                estimated_coordinates_with_ts = np.insert(estimated_coordinates_with_j, 0, ts)

            estimated_coordinates_with_ts_i = np.insert(estimated_coordinates_with_ts, 0, userid) # useridを列の先頭に追加
            i_res = np.vstack((i_res, estimated_coordinates_with_ts_i))

        measured_loc = np.vstack((measured_loc, i_res))
        measured_loc_df = pd.DataFrame(measured_loc, columns = ['userid', 'timestep', 'maxbeacon', 'x', 'y', 'z'])

    return measured_loc_df

##########################
###### 経路選択モデル ######
##########################

###### 経路選択モデリングのための時空間プリズム ######
### 個人の観測モデルのためのプリズムは別で用意すべしなはず ###
Ilist = []
#for i in range(L): # 今回はいらないよね．すでにIの設定の時点で与えてあるので
    #I[i, i] = 1 # Iは普通の空間接続行列．これに滞在リンクを追加している→階段自体にはないはず，その他の滞在リンクはすでにIの設定で実施済み
    
# 起点Oからの到達可能性指示変数ioの定義 kから出発して時刻（遷移回数）t以内に到達できるリンクaがII[t, k, a]=1になっている
Itlist = np.zeros((T+1, L, L))
II = np.copy(I)
Itlist[0, :, :] = np.eye(L) # ItlistにはT+1成分あるが，その第一成分(t=0)は単位行列ということ．t=1以降はfor内で計算

for ts in range(1, T+1): 
    Itlist[ts, :, :] = II # 時刻1では普通の接続行列
    II = np.dot(II, I) # 接続条件の積．つまり2回先に接続していたら1，そうでなければ0
    II = np.where(II > 0, 1, 0) 

# 終点Dからの到達可能性指示変数idの定義
Ittlist = np.zeros((T+1, L, L))
for ts in range(1, T+2): # Ittlist[T, ]までしか考えない
    Ittlist[ts-1, :, :] = np.transpose(Itlist[(T+1) - ts, :, :]) 

###### 即時効用行列 ######
def Mset(x): 
    cost = np.exp(df_link['length']/10 * x[0]) #  + df_link['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
    cost = pd.concat([cost]*L, axis=1)
    cost = cost.T
    cost_numpy = cost.values # DataFrameをNumPy配列に変換
    return cost_numpy

###### 対数尤度関数 ######
def loglikelihood(x):
    beta = x[-1]
    LL = 0
    
    # 観測モデル回し結果をuseridで分割
    measured_loc_df = measuring_model(x)

    grouped = measured_loc_df.groupby('userid')
    df_list = [group for name, group in grouped]
    df_list = [data.reset_index(drop=True) for data in df_list]    
    
    saikyo_key = [1, 2, 3, 4, 5, 6]
    yamate_key = [7, 8, 9, 10, 11, 12, 13]
    discriminate_key = [19, 22, 23]
    abs_beacons = [16, 20, 21]

    #### 以後個人ごとに処理（OD決定→個人プリズムの作成&経路選択モデル用のプリズムの作成→マッチング→経路データの作成→尤度関数の計算
    for i in range(len(df_list)):
        dfi = df_list[i]
        abs = dfi.loc[len(dfi)-1, 'maxbeacon']
        userid = dfi.loc[0, 'userid']
        maxbeacon_list = dfi['maxbeacon'].tolist()

        # 吸収がハチ公・中央以外だったら飛ばす
        if abs not in abs_beacons:
            continue
        
        # O決める
        first_beacon = int(dfi.loc[0, 'maxbeacon'])
        first_dist_list = d_array[:, first_beacon-1] 
        first_dist_dict = {index + 1: value for index, value in enumerate(first_dist_list)} # keyはlinkidになっている．max_beaconと各リンクの距離が入ってる

        if any(item in discriminate_key for item in maxbeacon_list): # 埼京オンリーにする
            filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in saikyo_key}
            max_key = min(filtered_dict, key = filtered_dict.get) # これが初期リンクとなる．埼京ホームのうち，最もmax_beaconに近いリンク
        else: 
            filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in yamate_key}
            max_key = min(filtered_dict, key = filtered_dict.get)
                  
        a0 = int(max_key) 
        tsi = len(dfi)

        # 次にDを確定
        last_beacon = int(dfi.loc[tsi-1, 'maxbeacon']) # 一番下の行を指定
        last_link = 0
        if last_beacon == 16: # node39
            last_link = 25 
            abs_link = 46
        elif last_beacon == 20: # node28
            last_link = 17
            abs_link = 47
        elif last_beacon == 21: # node29
            last_link = 18   
            abs_link = 48   
        else:
            continue  

        ###### OD決まったら時空間プリズム制約Idを入れる #####
        Id = np.zeros((T, L, L)) ######時空間プリズムでの最終タイムステップではなく，その個人の最終タイムステップtsiをプリズム制約に入れる必要がある．

        ao = a0 - 1 
        ad = abs_link -1 # last_link - 1      # ODlist.loc[od, 'd'] - 1

        for ts in range(T):
            if ts == 0:
                Id[ts, ao, :] = I[ao, :] # →全部利用可能なのおかしっくね，と思うが．
                continue

            if ts >= 1 and ts <= (len(dfi)):
                # ts>1の各時刻に対して
                # alistはOD（個人）に対して，時刻ts+1において
                alist = np.where(Itlist[ts + 1, ao, :] == Ittlist[ts + 1, ad, :])[0]  # aoからts+1で遷移可能なリンクと，adからts+1までに巻き戻し可能なリンクが共通するところ
                #### Itlist[ts + 1, ao, :]はaoをoとしたときに時刻ts+1に到達可能なリンクa，Ittlist[ts + 1, ad, :]はadをdとするときに時刻ts+1に滞在して良いリンク．これが共に1であるようなリンクをプリズム内に入れる（1にする），と言うのが本来の思想のはず．
                for a in alist:
                    if Itlist[ts + 1, ao, a] == Ittlist[ts + 1, ad, a]: # これelseなわけなくね，と思うが．
                        klist = np.where(I[:, a] == 1)[0] # このようなaに接続しているk

                        for k in klist:
                            if len(np.where(Id[ts - 1, :, k] == 1)[0]) != 0:
                                Id[ts, k, a] = 1 # ある個人が時刻tにkにいた時に利用可能なリンクaが1になっているはず．

            else: # ts >= len(dfi) and ts <= T
                Id[ts, :, :] = np.zeros((L, L))
                Id[ts, -3:, -3:] = 1 # 吸収リンクでの滞在のみ可能とする
        
        ##### 即時効用 #####
        M = np.zeros((T, L, L))
        for ts in range(T):
            Mts = Id[ts, :, :] * Mset(x)
            M[ts, :, :] = Mts

        ##### 価値関数 ##### 
        z = np.ones((T+1, L))
        for t in range(T, 0, -1):
            zii = M[t-1, :, :] * (z[t, :] ** beta) # Mt-1にプリズム条件が入り込んでいる
            zi = zii.sum(axis = 1)
            z[t-1, :] = (zi==0)*1 + (zi!=0)*zi
        
        print(z)

        dfi_data = pd.DataFrame({'t': range(1, T + 1)})
        # 'k', 'a' 列を追加し、初期化する
        dfi_data['k'] = 0
        dfi_data['a'] = 0
                    
        atk = a0 # 初期値

        for t in range(T): # len(dfi)を超えたら，吸収状態の行を追加していく
            
            ### 各時刻tにおいて，接続情報Idt[t]と観測結果dfi[t+1]を見て，その時の遷移先(ata=at+1)を決める．それをdfi_data[t, 'a']に追加．次のループでdfi_data[t, 'k']に追加すればいい
            ########### ata = at+1_k である．ここが勘違いの原因だった．
            ## t=0の時はdfi[0]からat=a0を決めていたが，dfi_data

            Idt = Id[t, :, :]

            if (t <= (len(dfi)-2)):
                dfi_data.loc[t, 'k'] = atk # tの時にいたリンクはatk
                
                a_row = Idt[atk-1, :]
                ata_cand = np.where(a_row == 1)[0]+1
                # print(f'user{userid}の時刻{t}での接続行列は{Idt}')

                # ここが多分違う．遷移先の位置を特定しようとしているのでt+1にする
                estx = dfi.loc[t+1, 'x']
                esty = dfi.loc[t+1, 'y']
                estz = dfi.loc[t+1, 'z']
                estloc = np.array([estx, esty, estz])

                res_dict = {} 

                if len(ata_cand) == 0:
                    break

                #print(f'user{userid}の時間制約は{len(dfi)}で時刻{t}での移動先の候補は{ata_cand}，今は{atk}')  #，移動自体は{tsi}までなので{tsi}ではkが{last_link}')

                for ata in ata_cand: # 次リンクataの中点位置(po+pd/2)とt+1でのestimated_locから計算する
                    # print(f'user{userid}の時刻{t}での移動先の候補は{at_cand}')
                    ##### 本当ならここでプリズムが効いてata_candがかなり絞られているはずなのだが
                    p_o = oddslink_loc_array[ata-1][0]
                    p_d = oddslink_loc_array[ata-1][0]
                    dist = shortest_distance_to_segment(p_o, p_d, estloc) # np.linalg.norm(est_loc - at_mid_loc)

                    pp = np.exp(np.log(M[t, atk-1, ata-1]) + beta * np.log(z[t+1, ata-1]) - np.log(z[t, atk-1]))

                    # ataの観測尤度
                    q = qq(dist)
                    
                    nume = q ** pp 
                    deno = 0
                        
                    a_prime_row = I[atk-1, :] # prime は，at_1からつながっている全リンク
                    ata_prime_list = np.where(a_prime_row == 1)[0] + 1                        

                    for ata_prime in ata_prime_list:
                        p_o_prime = oddslink_loc_array[ata_prime-1][0]
                        p_d_prime = oddslink_loc_array[ata_prime-1][1]
                        dist_prime = shortest_distance_to_segment(p_o_prime, p_d_prime, estloc) # np.linalg.norm(est_loc - at_mid_loc)
                        q_prime = qq(dist_prime)
                        pp = (np.exp(np.log(M[t, atk-1, ata_prime-1]) + beta * np.log(z[t+1, ata_prime-1]) - np.log(z[t, atk-1])))
                        deno += q_prime * pp
                    if deno == 0:
                        continue

                    delta = nume / deno 
                    # atにリンク尤度を入れる．これを辞書にする．valueが最大のものを次のat_1にする
                    # res_dict[at] = delta * np.log(pp) # 観測尤度を入れるのでdeltaだけ入る

                    res_dict[ata] = (delta, delta * np.log(pp))

                if not res_dict:
                    print('sss')
                    break

                else:
                    print(f'user{userid}時刻{t}の候補は{res_dict}')
                    max_delta_key = max(res_dict, key=lambda k: res_dict[k][0]) # deltaが最も大きいリンクのリンクidということ
                    dfi_data.loc[t, 'a'] = max_delta_key # これが次のdfi_data.loc[t, 'k']になると言うことじゃ！
                    # at_1 = max_delta_key # これはlinkid．毎回更新される  
                    atk = max_delta_key             # 次のatkなので
                    
                    #print(f'時刻{t}の次は{max_delta_key}へ移動')

                    #LL += res_dict[max_delta_key][1] # 最大リンクのdelta*np.log(pp)

            elif t == len(dfi)-1: # 最後→吸収の行をくわえる
                dfi_data.loc[t, 'k'] = last_link
                dfi_data.loc[t, 'a'] = abs_link
                #print(f'時刻{t}は{last_link}から{abs_link}')

            else: # t > len(dfi)の時は吸収→吸収だね
                dfi_data.loc[t, 'k'] = abs_link
                dfi_data.loc[t, 'a'] = abs_link                
                #print(f'時刻{t}は{abs_link}から{abs_link}')

        ####### ここまでで分析用にdfiを加工
        print(f'dfiは{dfi_data}')
        print(f'dfiは{dfi}')

        ##### 以上で時刻tにどのリンクを選んだかを確定的に推定し，以下で尤度計算に持ち込む ######
        klink = list(dfi_data.loc[:, 'k'])
        alink = list(dfi_data.loc[:, 'a'])

        for t in range(len(dfi)): # 吸収のところは実質的には関係ないので飛ばして良いのでは
            k = int(klink[t]) - 1 # at
            a = int(alink[t]) - 1 # at+1
            
            #pp = np.exp(np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k]))
            # print(f'個人{od+1}の時刻{t+1}の遷移は{k+1}から{a+1}で，その即時効用は{M[t, k, a]}，{a+1}の期待効用は{z[t+1, a]}')
            
            ##### 
            ##### 吸収の時はest_locとかの概念ない！！！！]
            """
            # t+1の観測を参照
            est_x = dfi.loc[t+1, 'x']
            est_y = dfi.loc[t+1, 'y']
            est_z = dfi.loc[t+1, 'z']
            est_loc = np.array([est_x, est_y, est_z])

            # q(estimated_x, selected_link) # linkの中点at_mid_locを取得する必要がある
            # 考えるのは遷移先のリンクのリンク尤度なのでa
            p_o = oddslink_loc_array[a][0]
            p_d = oddslink_loc_array[a][1]

            dist = shortest_distance_to_segment(p_o, p_d, est_loc) # np.linalg.norm(est_loc - at_mid_loc)

            # 一応観測方程式と呼べるものはこれか．
            q = qq(dist) # = dist/((sigma)**2) + np.exp(-((dist)**2)/2/((sigma)**2)) # レイリー分布

            k_row = I[k, :] # atからつながっているやつ
            a_prime_list = [] # at+1の候補集合
            a_prime_list = np.where(k_row == 1)[0]+1

            for a_prime in a_prime_list:
                p_o_prime = oddslink_loc_array[a_prime-1][0]
                p_d_prime = oddslink_loc_array[a_prime-1][1]
                dist_prime = shortest_distance_to_segment(p_o_prime, p_d_prime, est_loc) # np.linalg.norm(est_loc - at_mid_loc)
                q_prime = qq(dist_prime)
                        #dist = d_array[at_prime-1, max_beacon-1]
                        #pmat_prime = mq(dist, min_dist)
                pp = (np.exp(np.log(M[t, k, a_prime-1]) + beta * np.log(z[t+1, a_prime-1]) - np.log(z[t, k])))
                deno += q_prime * pp
            if deno == 0:
                continue
            """
            LL += np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])
            #delta = nume / deno 
            # 最後のリンクへの遷移が分かれば吸収への遷移とか吸収リンク内の遷移とかは確率考える必要なくね．
            #elif t == len(dfi)-1:
            #    delta = nume / deno
            logp = np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])
            ppp = np.exp(logp)
            #    LL += delta * logp
            print(f'user{userid}の時刻{t}の{k+1}から{a+1}への遷移,Mは{M[t, k, a]},ztは{z[t+1, a]}, zttは{z[t, k]}pppは{ppp}')

            #else:
            #    delta = nume / deno 
            #    logp = np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])

                #print(f'時刻{t}の{k}から{a}への遷移．deltaは{delta}, Mは{M[t, k, a]},ztは{z[t+1, a]}, zttは{z[t, k]}loppは{logp}')

            #    LL += delta * logp

    return -LL


In [18]:
x = np.array((-1, 0.6))
print(loglikelihood(x))
# 後ろの方は確かに吸収に接続するリンクがちゃんと効用高いが，それ以外が高くなってない．というかほぼ全てのリンクの効用が1で，接続してるっぽいリンクたちがそれより小さい値になってしまっている．
# これはいけないところが1似なってて，行けるところは下がっていくのかな．1のやつはそもそも家κないので関係ないと言うことか？
# 階段に行ってくれてない．その場にとどまるような選択になっている．
# 時空間プリズムがうまく機能してないのと，観測とリンクのマッチングがうまく機能していない可能性

パラメタ[-1.   0.6]で観測モデル回す
[[1.00000000e+00 1.00000000e+00 1.00000000e+00 6.11994051e-02
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 4.96375753e-02 4.84701252e-02
  3.56973050e-02 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+0

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:472: RuntimeWarning: divide by zero encountered in log
  pp = (np.exp(np.log(M[t, atk-1, ata_prime-1]) + beta * np.log(z[t+1, ata_prime-1]) - np.log(z[t, atk-1])))
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:481: RuntimeWarning: divide by zero encountered in log
  res_dict[ata] = (delta, delta * np.log(pp))
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot

[[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  4.99179881e-02 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 2.64582314e-02
  4.01615081e-02 4.04032437e-02 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.000

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:472: RuntimeWarning: divide by zero encountered in log
  pp = (np.exp(np.log(M[t, atk-1, ata_prime-1]) + beta * np.log(z[t+1, ata_prime-1]) - np.log(z[t, atk-1])))
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:481: RuntimeWarning: divide by zero encountered in log
  res_dict[ata] = (delta, delta * np.log(pp))
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:558: RuntimeWarning: divide by zero encountered in log
  LL += np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:563: RuntimeWarning: divide by zero encountered in log
  logp = np.log(M[t, k, a]) + b

[[1.00000000e+00 1.00000000e+00 1.00000000e+00 6.11994051e-02
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 4.96375753e-02 4.84701252e-02
  3.56973050e-02 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.000

...

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:472: RuntimeWarning: divide by zero encountered in log
  pp = (np.exp(np.log(M[t, atk-1, ata_prime-1]) + beta * np.log(z[t+1, ata_prime-1]) - np.log(z[t, atk-1])))
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:472: RuntimeWarning: divide by zero encountered in log
  pp = (np.exp(np.log(M[t, atk-1, ata_prime-1]) + beta * np.log(z[t+1, ata_prime-1]) - np.log(z[t, atk-1])))
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:45: RuntimeWarning


user1090.0の時刻0の3から2への遷移,Mは0.0820849986238988,ztは0.041909328778818256, zttは0.08148324214344499pppは0.15016956319658767
user1090.0の時刻1の2から2への遷移,Mは0.0820849986238988,ztは0.026707109550913495, zttは0.041909328778818256pppは0.2228072009355364
user1090.0の時刻2の2から3への遷移,Mは0.0820849986238988,ztは0.034860985991186366, zttは0.026707109550913495pppは0.41024255392012343
user1090.0の時刻3の3から3への遷移,Mは0.0820849986238988,ztは0.016134427028828828, zttは0.034860985991186366pppは0.19795945853368774
user1090.0の時刻4の3から3への遷移,Mは0.0820849986238988,ztは0.005195304718705231, zttは0.016134427028828828pppは0.2167089576144387
user1090.0の時刻5の3から38への遷移,Mは0.0820849986238988,ztは0.010051835744633584, zttは0.005195304718705231pppは1.0
user1090.0の時刻6の38から39への遷移,Mは0.0820849986238988,ztは0.030197383422318504, zttは0.010051835744633584pppは1.0
user1090.0の時刻7の17から47への遷移,Mは1.0,ztは2.8710212339441608, zttは2.1673713729447694pppは0.8687373823432247
[[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 3.60151388e

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:472: RuntimeWarning: divide by zero encountered in log
  pp = (np.exp(np.log(M[t, atk-1, ata_prime-1]) + beta * np.log(z[t+1, ata_prime-1]) - np.log(z[t, atk-1])))
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:481: RuntimeWarning: divide by zero encountered in log
  res_dict[ata] = (delta, delta * np.log(pp))
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:472: RuntimeWarning: divide by zero encountered in log
  pp = (np.exp(np.log(M[t, atk-1, ata_prime-1]) + beta * np.log(z[t+1, ata_prime-1]) - np.log(z[t, atk-1])))
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:45: RuntimeWarning: invalid value encountered in 

[[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  5.36136407e-02 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 5.44829402e-02
  5.41197913e-02 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.000

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:558: RuntimeWarning: divide by zero encountered in log
  LL += np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1769/1234877129.py:563: RuntimeWarning: divide by zero encountered in log
  logp = np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])
